## IMPORT LIBRARIES

### FOR SPEECH RECOGNITION

In [1]:
import speech_recognition as sr
import pytz
from google.cloud import texttospeech
import os
from gtts import gTTS
import pyttsx3
import playsound
import subprocess

### FOR NLTK DEPENDENCY TREE

In [2]:
import spacy
from spacy import displacy
from spacy.symbols import NOUN, NUM

### FOR GOOGLE CALENDAR

In [3]:
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

### GLOBAL VARIABLES

In [4]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

DAYS = ["monday", "tuesday", "wednesday","thursday","friday","saturday","sunday"]
MONTHS = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
DAY_EXTENTIONS = ["rd","th","st"]

CALENDAR_STRS = ["what do i have", "do i have plans", "am i busy"]
NOTE_STRS = ["make a note", "write this down", "remember this"]

TALKTOHER = "hello"

### SEE VERSIONS

print(sr.__version__)
print(spacy.__version__)
print(nltk.__version__)

### GETTING SPEECH

In [5]:
def speak(text):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate - 50)
    engine.setProperty("volume", 1.0)
    engine.say(text)
    engine.runAndWait()

### GETTING AUDIO

In [6]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))
            speak("Sorry, I could not understand what you said.")

        return said.lower()

### CHECKING SENTENCE

In [7]:
def check(sentence, words):
    res = [all([k in s for k in words]) for s in sentence]
    return [sentence[i] for i in range(0, len(res)) if res[i]]

### GOOGLE AUTHENTICATION

In [ ]:
def googleAuthentication():
    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'api/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    return service

### GETTING EVENTS FROM GOOGLE

In [ ]:
def getEventsfromGoogle(day, service): #n is amount of events
    # Call the Calendar API

    date = datetime.datetime.combine(day, datetime.datetime.min.time())
    end_date = datetime.datetime.combine(day, datetime.datetime.max.time())
    utc = pytz.UTC
    date = date.astimezone(utc)
    end_date = end_date.astimezone(utc)
    events_result = service.events().list(calendarId ='primary', timeMin = date.isoformat(), timeMax = end_date.isoformat(),
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        speak('No upcoming events found.')
    else:
        speak(f"You have {len(events)} event on this day.")
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])
            start_time = str(start.split("T")[1].split("-")[0])
            if int(start_time.split(":")[0]) < 12:
                start_time = start_time + "am"
            else:
                start_time = str(int(start_time.split(":")[0]) - 12) + start_time.split(":")[1]
                start_time = start_time + "pm"

            speak(event["summary"] + " at " + start_time)

### GETTING DATES

In [ ]:
def get_date(text):
    text = text.lower()
    today = datetime.date.today()

    if text.count("today") > 0:
        return today

    day = -1
    day_of_week = -1
    month = -1
    year = today.year

    for word in text.split():
        if word in MONTHS:
            month = MONTHS.index(word) + 1
        elif word in DAYS:
            day_of_week = DAYS.index(word)
        elif word.isdigit():
            day = int(word)
        else:
            for ext in DAY_EXTENTIONS:
                found = word.find(ext)
                if found > 0:
                    try:
                        day = int(word[:found])
                    except:
                        pass
    if month < today.month and month != -1:
        year = year + 1
    if day < today.day and month == -1 and day != -1:
        month = month + 1
    if month == -1 and day == -1 and day_of_week != -1:
        current_day_of_week = today.weekday()
        dif = day_of_week - current_day_of_week

        if dif < 0:
            dif += 7
            if text.count("next") >= 1:
                dif += 7
        return today + datetime.timedelta(dif)
    if month == -1 or day == -1:
        return None

    return datetime.date(month = month, day = day, year = year)

### WRITING INTO NOTEPAD/CALENDAR

In [ ]:
def note(text):
    date = datetime.datetime.now()
    file_name = str(date).replace(":", "-") + "-note.txt"
    with open(file_name, "w") as f:
        f.write(text)

    subprocess.Popen(["notepad.exe", file_name])
    

### START GOOGLE AUTHENTICATION

In [ ]:
SERVICE = googleAuthentication()

In [ ]:
### START THE PROCESS

In [ ]:
while True:
    text = get_audio()

    if text.count(TALKTOHER) > 0:
        speak("Hello, how may I help you today?")
        text = get_audio()

        for phrase in CALENDAR_STRS:
            if phrase in text.lower():
                date = get_date(text)
                if date:
                    getEventsfromGoogle(get_date(text), SERVICE)
                else:
                    speak("Sorry, I didn't understand what you have said. Could you please repeat?")

        for phrase in NOTE_STRS:
            if phrase in NOTE_STRS:
                if phrase in text.lower():
                    speak("What would you like me to write down?")
                    note_text = get_audio().lower()
                    note(note_text)
                    speak("I have made a note of that.")

        if "name" in text.split():
            speak("My name is Alice and I'm here to help you with booking.")


        if "Malta" in text.split():
            speak("Yes. She absolutely needs to sleep now. Nijat cares for her.")


hello there
